In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4193/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_18_8_2,0.997499,0.787661,0.999698,0.984578,0.996479,0.016722,1.419910,4.782672e-03,0.066194,0.035488,0.206128,0.129312,1.001819,0.134817,122.182119,191.658041,"Hidden Size=[14], regularizer=0.5, learning_ra..."
1,model_18_8_3,0.997329,0.782684,0.999219,0.978836,0.994879,0.017859,1.453190,1.239200e-02,0.090838,0.051615,0.159797,0.133636,1.001942,0.139326,122.050530,191.526452,"Hidden Size=[14], regularizer=0.5, learning_ra..."
11,model_18_9_1,0.996537,0.777213,0.996787,0.998836,0.998280,0.023154,1.489775,1.382200e-02,0.008142,0.010982,0.107320,0.152166,1.002518,0.158644,121.531143,191.007065,"Hidden Size=[14], regularizer=0.5, learning_ra..."
24,model_18_9_0,0.996536,0.777211,0.996803,0.998836,0.998285,0.023166,1.489789,1.375437e-02,0.008144,0.010949,0.106666,0.152204,1.002520,0.158683,121.530138,191.006060,"Hidden Size=[14], regularizer=0.5, learning_ra..."
25,model_18_8_1,0.996087,0.796230,1.000000,0.993122,0.998536,0.026164,1.362611,1.313099e-08,0.029523,0.014761,0.281666,0.161754,1.002846,0.168640,121.286721,190.762643,"Hidden Size=[14], regularizer=0.5, learning_ra..."
26,model_18_8_0,0.996086,0.796235,1.000000,0.993130,0.998537,0.026176,1.362579,4.704167e-13,0.029485,0.014743,0.281338,0.161789,1.002847,0.168677,121.285852,190.761774,"Hidden Size=[14], regularizer=0.5, learning_ra..."
45,model_13_9_6,0.995002,0.783371,0.971405,0.992499,0.993392,0.033419,1.448596,1.379907e-02,0.091616,0.052708,0.462113,0.182809,1.004136,0.190592,112.797244,177.397663,"Hidden Size=[13], regularizer=0.5, learning_ra..."
47,model_13_9_5,0.994647,0.784208,0.973995,0.993170,0.993984,0.035797,1.443004,1.254927e-02,0.083420,0.047985,0.480772,0.189202,1.004430,0.197256,112.659767,177.260186,"Hidden Size=[13], regularizer=0.5, learning_ra..."
48,model_13_9_4,0.994165,0.785060,0.976729,0.993873,0.994605,0.039015,1.437304,1.123008e-02,0.074826,0.043028,0.501654,0.197523,1.004829,0.205932,112.487596,177.088015,"Hidden Size=[13], regularizer=0.5, learning_ra..."
49,model_13_9_3,0.993524,0.785912,0.979588,0.994604,0.995251,0.043302,1.431604,9.850179e-03,0.065905,0.037877,0.525063,0.208091,1.005359,0.216950,112.279111,176.879530,"Hidden Size=[13], regularizer=0.5, learning_ra..."
